# Sistema de Recomendación de Canciones

![Music4U](Music4U_logo.png)

**Importamos las librerías necesarias**

In [41]:
#Importamos librerias necesarias
import pandas as pd
import zipfile as zp
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import widgets, Button, interact_manual, Layout
import plotly.offline as py 
from plotly.figure_factory import create_table
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from IPython.display import display, clear_output

**Importamos la tabla de canciones y la de artistas con géneros, ya que serán las dos tablas de la base de datos de Spotify que usaremos para este sistema de recomendación**

In [2]:
#Importar las tablas de la base de datos
archive = zp.ZipFile("archive.zip", "r")
original_df = pd.read_csv(archive.open("data.csv"))
original_df_genero = pd.read_csv(archive.open("data_w_genres.csv"))

**Preparamos los datos en la tabla de canciones**

In [3]:
#Convierte las listas de string en la columna de artistas en listas de verdad
original_df['artists'] = original_df['artists'].apply(lambda x: x[1:-1].split(', ')) 
#Separar las listas para que cada artistas tenga una fila con una cancion
original_df = original_df.explode('artists')
#Quitamos las comillas a las listas de la columna de artistas
original_df['artists'] = original_df['artists'].apply(lambda x: x.strip("'"))

#Renombramos algunas columnas
df = original_df.rename(columns={'name':'Cancion', 'popularity':'Popularidad Cancion', 'year':'Anio'}, inplace=True)

#Guardamos solomente las siguientes columnas en una nueva tabla
df = original_df.loc[:,['artists', 'Cancion', 'Popularidad Cancion', 'Anio']]
df.head(3)

,artists,Cancion,Popularidad Cancion,Anio
0,Carl Woitschach,Singende Bataillone 1. Teil,0,1928
1,Robert Schumann,"Fantasiestücke, Op. 111: Più tosto lento",0,1928
1,Vladimir Horowitz,"Fantasiestücke, Op. 111: Più tosto lento",0,1928


In [4]:
#Creamos una columna nuevas que contenga las decadas de las canciones
df['Decada Cancion'] = None

df.loc[(df['Anio'] >= 1920) & (df['Anio'] < 1930), 'Decada Cancion'] = '1920s'
df.loc[(df['Anio'] >= 1930) & (df['Anio'] < 1940), 'Decada Cancion'] = '1930s'
df.loc[(df['Anio'] >= 1940) & (df['Anio'] < 1950), 'Decada Cancion'] = '1940s'
df.loc[(df['Anio'] >= 1950) & (df['Anio'] < 1960), 'Decada Cancion'] = '1950s'
df.loc[(df['Anio'] >= 1960) & (df['Anio'] < 1970), 'Decada Cancion'] = '1960s'
df.loc[(df['Anio'] >= 1970) & (df['Anio'] < 1980), 'Decada Cancion'] = '1970s'
df.loc[(df['Anio'] >= 1980) & (df['Anio'] < 1990), 'Decada Cancion'] = '1980s'
df.loc[(df['Anio'] >= 1990) & (df['Anio'] < 2000), 'Decada Cancion'] = '1990s'
df.loc[(df['Anio'] >= 2000) & (df['Anio'] < 2010), 'Decada Cancion'] = '2000s'
df.loc[(df['Anio'] >= 2010) & (df['Anio'] < 2020), 'Decada Cancion'] = '2010s'
df.loc[(df['Anio'] >= 2020) & (df['Anio'] < 2030), 'Decada Cancion'] = '2020s'
df.head(3)

,artists,Cancion,Popularidad Cancion,Anio,Decada Cancion
0,Carl Woitschach,Singende Bataillone 1. Teil,0,1928,1920s
1,Robert Schumann,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,1920s
1,Vladimir Horowitz,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,1920s


**Preparar los datos en la tabla de géneros**

In [5]:
#Eliminamos los generos nulos
no_generos_nulos = original_df_genero[original_df_genero.genres != "[]"]
#Creamos una nueva tabla que solo contenga la columna de generos, artistas y popularidad, 
#ya que estas son las unicas columnas que nos interesan
df_genero = no_generos_nulos.loc[:, ['artists', 'genres', 'popularity']]
#Convertir la popularidad de float a entero
df_genero['popularity'] = df_genero['popularity'].astype(int)
df_genero = df_genero.reset_index(drop=True)
#Convertir las listas de string en la columna de generos en listas de verdad
df_genero['genres'] = df_genero['genres'].apply(lambda x: x[1:-1].split(', '))
#Renombrar la columna de popularidad
df_genero.rename(columns={'popularity':'Popularidad Artista'}, inplace=True)
df_genero.head(3)

,artists,genres,Popularidad Artista
0,"""Cats"" 1981 Original London Cast",['show tunes'],38
1,"""Weird Al"" Yankovic","['antiviral pop', 'comedy rock', 'comic', 'par...",33
2,$NOT,"['east coast hip hop', 'gangster rap', 'hardco...",64


In [6]:
#Fusionar tabla de canciones con la tabla de generos
df_fusionado = original_df.merge(df_genero, how='inner', on = ['artists'])
#Eliminamos las columnas que no necesitamos
df_fusionado = df_fusionado.drop(['duration_ms', 'explicit', 'id', 'release_date'], axis=1)
#Renombramos la columna de popularidad cancion
df_fusionado.rename(columns={'popularity': 'Popularidad Cancion'}, inplace=True)
df_fusionado.head(3)

,acousticness,artists,danceability,energy,instrumentalness,key,liveness,loudness,mode,Cancion,Popularidad Cancion,speechiness,tempo,valence,Anio,genres,Popularidad Artista
0,0.994,Robert Schumann,0.379,0.01350,0.901,8,0.0763,-28.454,1,"Fantasiestücke, Op. 111: Più tosto lento",0,0.0462,83.972,0.0767,1928,"['classical', 'early romantic era']",3
1,0.992,Robert Schumann,0.311,0.01070,0.883,5,0.0954,-35.648,1,"Nachtstücke, Op. 23: No. 4 in F",0,0.0556,78.980,0.2160,1928,"['classical', 'early romantic era']",3
2,0.995,Robert Schumann,0.319,0.00217,0.937,10,0.0791,-36.905,1,"Humoreske, Op. 20: Einfach",0,0.0412,78.505,0.2290,1928,"['classical', 'early romantic era']",3


In [7]:
#Creamos una columna para la decada de la cancion
df_fusionado['Decada Cancion'] = None

df_fusionado.loc[(df_fusionado['Anio'] >= 1920) & (df_fusionado['Anio'] < 1930), 'Decada Cancion'] = '1920s'
df_fusionado.loc[(df_fusionado['Anio'] >= 1930) & (df_fusionado['Anio'] < 1940), 'Decada Cancion'] = '1930s'
df_fusionado.loc[(df_fusionado['Anio'] >= 1940) & (df_fusionado['Anio'] < 1950), 'Decada Cancion'] = '1940s'
df_fusionado.loc[(df_fusionado['Anio'] >= 1950) & (df_fusionado['Anio'] < 1960), 'Decada Cancion'] = '1950s'
df_fusionado.loc[(df_fusionado['Anio'] >= 1960) & (df_fusionado['Anio'] < 1970), 'Decada Cancion'] = '1960s'
df_fusionado.loc[(df_fusionado['Anio'] >= 1970) & (df_fusionado['Anio'] < 1980), 'Decada Cancion'] = '1970s'
df_fusionado.loc[(df_fusionado['Anio'] >= 1980) & (df_fusionado['Anio'] < 1990), 'Decada Cancion'] = '1980s'
df_fusionado.loc[(df_fusionado['Anio'] >= 1990) & (df_fusionado['Anio'] < 2000), 'Decada Cancion'] = '1990s'
df_fusionado.loc[(df_fusionado['Anio'] >= 2000) & (df_fusionado['Anio'] < 2010), 'Decada Cancion'] = '2000s'
df_fusionado.loc[(df_fusionado['Anio'] >= 2010) & (df_fusionado['Anio'] < 2020), 'Decada Cancion'] = '2010s'
df_fusionado.loc[(df_fusionado['Anio'] >= 2020) & (df_fusionado['Anio'] < 2030), 'Decada Cancion'] = '2020s'

df_fusionado.head(2)

,acousticness,artists,danceability,energy,instrumentalness,key,liveness,loudness,mode,Cancion,Popularidad Cancion,speechiness,tempo,valence,Anio,genres,Popularidad Artista,Decada Cancion
0,0.994,Robert Schumann,0.379,0.0135,0.901,8,0.0763,-28.454,1,"Fantasiestücke, Op. 111: Più tosto lento",0,0.0462,83.972,0.0767,1928,"['classical', 'early romantic era']",3,1920s
1,0.992,Robert Schumann,0.311,0.0107,0.883,5,0.0954,-35.648,1,"Nachtstücke, Op. 23: No. 4 in F",0,0.0556,78.980,0.2160,1928,"['classical', 'early romantic era']",3,1920s


In [8]:
#Cambiamos el nombre de la columna artists a Artista
#df_fusionado.rename(columns={'artists':'Artista'}, inplace=True)
#Reordenamos las columnas para que se puedan visualizar los datos de una mejor manera
columns_reorder = ['artists', 'Cancion', 'Popularidad Cancion', 'Anio', 'genres', 'Popularidad Artista',
                   'Decada Cancion', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'valence']

df_fusionado = df_fusionado.reindex(columns=columns_reorder)
df_fusionado.head(2)

,artists,Cancion,Popularidad Cancion,Anio,genres,Popularidad Artista,Decada Cancion,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,Robert Schumann,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,"['classical', 'early romantic era']",3,1920s,0.994,0.379,0.0135,0.901,8,0.0763,-28.454,1,0.0462,83.972,0.0767
1,Robert Schumann,"Nachtstücke, Op. 23: No. 4 in F",0,1928,"['classical', 'early romantic era']",3,1920s,0.992,0.311,0.0107,0.883,5,0.0954,-35.648,1,0.0556,78.980,0.2160


In [9]:
#Separamos las columnas musicales numericas en una nueva tabla
datos_cancion = df_fusionado.loc[:, ['acousticness', 'danceability', 'energy', 'instrumentalness', 
                              'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
                             'valence']]

#MinMaxScaler: Transforma las características escalando cada característica a un rango determinado.
#Este estimador escala y traduce cada característica individualmente de manera que se encuentre en el 
#rango dado en el conjunto de entrenamiento, en nuestro caso de -1 a 1.
scaler = MinMaxScaler()

caracteristicas_cancion = pd.DataFrame()

for col in datos_cancion.iloc[:, :].columns:
    scaler.fit(datos_cancion[[col]])
    caracteristicas_cancion[col] = scaler.transform(datos_cancion[col].values.reshape(-1,1)).ravel()

caracteristicas_cancion.head(2)

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,0.997992,0.383603,0.0135,0.901,0.727273,0.0763,0.494886,1.0,0.047826,0.344019,0.0767
1,0.995984,0.314777,0.0107,0.883,0.454545,0.0954,0.382028,1.0,0.057557,0.323568,0.2160


In [10]:
#Reemplazar los viejos valores por los nuevos valores ya escalados
reemplazar_datos = df_fusionado.drop(['acousticness', 'danceability', 'energy', 'instrumentalness', 
                              'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
                             'valence'], axis=1)
                               
df_final = reemplazar_datos.join(caracteristicas_cancion)
df_final.head(2)

,artists,Cancion,Popularidad Cancion,Anio,genres,Popularidad Artista,Decada Cancion,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
0,Robert Schumann,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,"['classical', 'early romantic era']",3,1920s,0.997992,0.383603,0.0135,0.901,0.727273,0.0763,0.494886,1.0,0.047826,0.344019,0.0767
1,Robert Schumann,"Nachtstücke, Op. 23: No. 4 in F",0,1928,"['classical', 'early romantic era']",3,1920s,0.995984,0.314777,0.0107,0.883,0.454545,0.0954,0.382028,1.0,0.057557,0.323568,0.2160


**Ya que los datos estan preparados procedemos a realizar nuestro sistema de recomendacion de canciones**

In [38]:
def sistema_recomendacion_cancion(boton):
    with outuser_info:
        clear_output()
        
        #Verificamos que el artista y la cancion ingresada existan en la tabla (ya fusinada)
        datos_artista_cancion = df_final[(df_final['artists'] == artista_txtbox.value) & (df_final['Cancion'] == cancion_txtbox.value)].sort_values('Anio')[0:1]

        #Hacemos una copia de la tabla original para poder trabajar sobre ella
        similitud_datos = df_final.copy()

        #Obtenemos solamente los datos musicales numericos, pues con estos trabajaremos para calcular la similitud del coseno
        valores_datos = similitud_datos.loc[:, ['acousticness', 'danceability', 'energy', 'instrumentalness', 
                                      'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
                                     'valence']]

        #Creamos una nueva columna que se llame 'Similitud con Artista'
        #Posteriormente aqui realizaremos la similitud del coseno con los datos musicales numericos que extrajimos
        #La similitud del coseno se aplica a todos las canciones
        #similitud_datos = pd.DataFrame(index=[0])
        similitud_datos['Similitud con Cancion'] = cosine_similarity(valores_datos, valores_datos.to_numpy()[datos_artista_cancion.index[0], None]).squeeze()

        #Obtenemos los generos que contiene ese artista (es una lista)   
        generos_artista = set(*datos_artista_cancion.genres)

        #Buscamos una interseccion entre todos los generos de la tabla con los generos de nuestro artista ingresado 
        similitud_datos.genres = similitud_datos.genres.apply(lambda genres: list(set(genres).intersection(generos_artista)))

        longitudes_similitud = similitud_datos.genres.str.len()
        #Dado el parametro del genero esta variable obtendra mas generos o el mismo numero de generos que deseemos 
        similitud_datos = similitud_datos.reindex(longitudes_similitud[longitudes_similitud >= genero_slider.value].sort_values(ascending=False).index)

        #Nos aseguramos de que la decada de la cancion ingresada coincida con la decada de las canciones similares
        similitud_datos = similitud_datos[similitud_datos['Decada Cancion'] == datos_artista_cancion['Decada Cancion'].values[0]]

        #Acomodamos los datos de mayor a menor
        similitud_datos = similitud_datos.sort_values(by = 'Similitud con Cancion', ascending=False)
        
        #Renombramos algunas columnas
        similitud_datos.rename(columns={'artists':'Artista', 'Cancion':'Cancion Similar', 'genres':'Generos'}, inplace=True)
        
        #Creamos una tabla que solo contenga los primeros 10 elementos para trabajar con ella
        tabla_grafica = similitud_datos.head(10)
        
        #Creamos una grafica con estos nuevos datos que acabamos de obtener
        size = [100, 90, 80, 70, 60, 50, 40, 30, 20, 10]
        fig = go.Figure(data=[go.Scatter(
            x=tabla_grafica['Similitud con Cancion'], y=tabla_grafica['Cancion Similar'], 
            text=tabla_grafica['Artista'],
            mode='markers',
            marker=dict(
                color=['rgb(255, 241, 0)', 'rgb(255, 140, 0)', 'rgb(232, 17, 35)', 'rgb(236, 0, 140)',
                      'rgb(104, 33, 122)', 'rgb(0, 24, 143)', 'rgb(0, 188, 242)', 'rgb(0, 178, 148)',
                      'rgb(0, 158, 73)', 'rgb(186, 216, 10)'],
                size=size,
            ),
        )])
        fig.update_layout(title='Canciones similares a ' + cancion_txtbox.value)

        display(similitud_datos.loc[:, ['Artista', 'Cancion Similar', 'Popularidad Cancion', 'Anio', 'Generos', 'Popularidad Artista', 'Similitud con Cancion']].head(10))
        fig.show()

In [46]:
#Textbox para ingresar la cancion
cancion_txtbox = widgets.Text(  
     placeholder = 'Red',  
     description = 'Cancion:',
     value = 'Red',
) 
#Textbox para ingresar el artista
artista_txtbox = widgets.Text(  
     placeholder = 'Taylor Swift',  
     description = 'Artista:',
     value = 'Taylor Swift',
)  
style = {'description_width': 'initial'}
#Slider para ingresar el minimo de generos que coincidan
genero_slider = widgets.IntSlider(description='Numero de generos', min = 1, max=5, style=style, value=1)

#Creamos el boton que presionaremos para obtener el sistema de recomendacion
boton_recomendar = Button(description='Recomendar Canciones', layout=Layout(width='18%'))
#Esto hara que cada vez que presionemos de nuevo el boton el output antiguo sea borrado
outuser_info = widgets.Output()

#Indicamos que queremos cada vez que se de clic al boton, se llame a la funcion que realiza el sistema de recomendacion
boton_recomendar.on_click(sistema_recomendacion_cancion)

#Mostramos el textbox, el slider y el boton
display(cancion_txtbox, artista_txtbox, genero_slider, boton_recomendar, outuser_info) 

Text(value='Red', description='Cancion:', placeholder='Red')

Text(value='Taylor Swift', description='Artista:', placeholder='Taylor Swift')

IntSlider(value=1, description='Numero de generos', max=5, min=1, style=SliderStyle(description_width='initial…

Button(description='Recomendar Canciones', layout=Layout(width='18%'), style=ButtonStyle())

Output()